In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [5]:
df=pd.read_csv('SMSSpamCollection.txt',sep="\t",names=["label","message"])

In [8]:
df


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
from nltk.corpus import stopwords
sw=stopwords.words('english')

In [33]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
corpus=[]
for i in range(0,len(df)):
    sentence=df.iloc[i,1]
    sentence=sentence.lower()
    sentence=sentence.split()
    newSentence=[lemmatizer.lemmatize(word) for word in sentence if word not in sw]
    newSentence=" ".join(newSentence)
    corpus.append(newSentence)

In [34]:
print(corpus)

['go jurong point, crazy.. available bugis n great world la e buffet... cine got amore wat...', 'ok lar... joking wif u oni...', "free entry 2 wkly comp win fa cup final tkts 21st may 2005. text fa 87121 receive entry question(std txt rate)t&c's apply 08452810075over18's", 'u dun say early hor... u c already say...', 'nah think go usf, life around though', "freemsg hey darling 3 week's word back! like fun still? tb ok! xxx std chgs send, £1.50 rcv", 'even brother like speak me. treat like aid patent.', "per request 'melle melle (oru minnaminunginte nurungu vettam)' set callertune callers. press *9 copy friend callertune", 'winner!! valued network customer selected receivea £900 prize reward! claim call 09061701461. claim code kl341. valid 12 hour only.', 'mobile 11 month more? u r entitled update latest colour mobile camera free! call mobile update co free 08002986030', 'gonna home soon want talk stuff anymore tonight, k? cried enough today.', 'six chance win cash! 100 20,000 pound txt

In [49]:
y=pd.get_dummies(df,columns=['label'],dtype=int)

In [50]:
y=y.iloc[:,1]

In [51]:
y   #ham->1
    #spam->0

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: label_ham, Length: 5572, dtype: int32

In [87]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(corpus,y,test_size=0.2,random_state=42)

Bag of Words:

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
x_train=vectorizer.fit_transform(x_train)
x_test=vectorizer.transform(x_test)

In [84]:
x_train=x_train.toarray()
x_test=x_test.toarray()

In [85]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
model=MultinomialNB()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
print(classification_report(y_true=y_test,y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.97      0.94      0.95       149
           1       0.99      0.99      0.99       966

    accuracy                           0.99      1115
   macro avg       0.98      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115



TF-IDF:

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
x_train=vectorizer.fit_transform(x_train)
x_test=vectorizer.transform(x_test)


In [89]:
x_train=x_train.toarray()
x_test=x_test.toarray()

In [90]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
model=MultinomialNB()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
print(classification_report(y_true=y_test,y_pred=y_pred))

              precision    recall  f1-score   support

           0       1.00      0.81      0.89       149
           1       0.97      1.00      0.99       966

    accuracy                           0.97      1115
   macro avg       0.99      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115



Word2Vec:

In [95]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')


In [138]:
corpus

['go jurong point, crazy.. available bugis n great world la e buffet... cine got amore wat...',
 'ok lar... joking wif u oni...',
 "free entry 2 wkly comp win fa cup final tkts 21st may 2005. text fa 87121 receive entry question(std txt rate)t&c's apply 08452810075over18's",
 'u dun say early hor... u c already say...',
 'nah think go usf, life around though',
 "freemsg hey darling 3 week's word back! like fun still? tb ok! xxx std chgs send, £1.50 rcv",
 'even brother like speak me. treat like aid patent.',
 "per request 'melle melle (oru minnaminunginte nurungu vettam)' set callertune callers. press *9 copy friend callertune",
 'winner!! valued network customer selected receivea £900 prize reward! claim call 09061701461. claim code kl341. valid 12 hour only.',
 'mobile 11 month more? u r entitled update latest colour mobile camera free! call mobile update co free 08002986030',
 'gonna home soon want talk stuff anymore tonight, k? cried enough today.',
 'six chance win cash! 100 20,00

In [155]:
x=[]
for i in corpus:
    sentence=i
    words=sentence.split()
    temp=[]
    for word in words:
        if word in wv:
            temp.append(wv[word])
        else:
            temp.append(np.zeros(300))
    temp=np.mean(temp,axis=0)
    x.append(temp)
    

In [156]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [157]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
model=RandomForestClassifier()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
print(classification_report(y_true=y_test,y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.98      0.74      0.84       149
           1       0.96      1.00      0.98       966

    accuracy                           0.96      1115
   macro avg       0.97      0.87      0.91      1115
weighted avg       0.96      0.96      0.96      1115

